In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import random
import re
import json
from collections import Counter
import copy

In [2]:
from utils import tf_lower_and_remove_punct,tf_lower_and_remove_punct_1
from utils import masked_loss,masked_acc,beam_search_decoder
from utils import Transformer,CustomSchedule

In [3]:
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [4]:
df = pd.read_csv('data/english_hindi_10k_vocab.csv',usecols = ['english','hindi'])
df.head()

,english,hindi
0,we do the same too .,हम भी ऐसा ही करेंगे .
1,these will take a couple of days more .,इसमें एक दो दिन और लग जाएंगे ।
2,how much ?,. कितना ?
3,new search folder,नया खोज फ़ोल्डर
4,"life might be listening , and give you less th...","अगर जिंदगी ने कहीं सुन लिया , तो अगली बार इससे..."


In [5]:
text_pairs = list(zip(df['english'],df['hindi']))
random.seed(42)
random.shuffle(text_pairs)

# Assigning  data for validation and testing set
num_val_samples = int(0.1*len(text_pairs))
num_test_samples = int(0.05*len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples - num_test_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples+num_val_samples]
test_pairs = text_pairs[num_train_samples+num_val_samples:]
print(f"length of text_pairs  {len(text_pairs)}")
print(f"length of train_pairs {len(train_pairs)}")
print(f"length of val_pairs   {len(val_pairs)}")
print(f"length of test_pairs  {len(test_pairs)}")

length of text_pairs  181602
length of train_pairs 154362
length of val_pairs   18160
length of test_pairs  9080


In [6]:
with open("data/customized_val_pairs.txt",'r',encoding = 'utf-8') as f:
    customized_val_pairs = f.readlines()
    customized_val_pairs = [tuple(i.strip('\n').split('\t')) for i in customized_val_pairs]
customized_val_pairs[:10]

[('force instant hide', 'त्वरित छुपाना बाध्य करें'),
 ('pink marble', 'गुलाबी संगमरमर'),
 ('i am happy with her performance .', 'मैं उसके प्रदर्शन से खुश हूं ।'),
 ('kolab', 'कोलाब'),
 ('modify task cost', 'कार्य लागत परिवर्धित करें'),
 ('color mode', 'रंग विधि'),
 ('bird tweet', 'चिड़ियों की चहचहाहट'),
 ('canadian aboriginal', 'कनाडियन एबॉर्जिनल'),
 ('it s rather cold today .', 'आज मौसम काफ़ी ठंडा है ।'),
 ('calendar format', 'कैलेन्डर फॉर्मेट')]

In [7]:
with open('Text_vectorization_file/english_vocab.json','r',encoding = 'utf-8') as f:
    english_vocab = json.load(f)

with open('Text_vectorization_file/hindi_vocab.json','r',encoding = 'utf-8') as f:
    hindi_vocab = json.load(f)

In [8]:
max_vocab_size =15000
sequence_length = 25
#Recreate the English vectorization layer with basic configuration
english_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = max_vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length,
    standardize = tf_lower_and_remove_punct
)


hindi_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = max_vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length+1,
    standardize = tf_lower_and_remove_punct_1
)

english_vectorization.set_vocabulary(english_vocab)
hindi_vectorization.set_vocabulary(hindi_vocab)

In [9]:
print(len(english_vocab))
print(len(hindi_vocab))

10438
11697


In [10]:
batch_size = 64
def format_dataset(eng,hin):
    eng = english_vectorization(eng)
    hin = hindi_vectorization(hin)
    tar_in = hin[:,:-1]
    tar_out = hin[:,1:]
    return (eng,tar_in),tar_out

def make_dataset(pairs):
    eng_texts, hin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    hin_texts = list(hin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, hin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset

new_val_ds = make_dataset(customized_val_pairs)


In [11]:
print(len(new_val_ds))

606


In [12]:
for (inp,tar_in),tar_out in new_val_ds.take(1):
    pass
print(inp.shape)
print(tar_in.shape)
print(tar_out.shape)

(64, 25)
(64, 25)
(64, 25)


In [13]:
embed_dim = 320
heads = 2
latent = 2048
no_of_layers = 4
vocab_size_eng = len(english_vocab)
vocab_size_hin = len(hindi_vocab)

transformer1 = Transformer(embedding_dim=embed_dim,num_heads=heads,dense_dim=latent,
                          num_layers=no_of_layers,input_vocab_size=vocab_size_eng,
                           output_vocab_size=vocab_size_hin)

In [14]:
logits = transformer1((inp,tar_in))
transformer1.summary()
learning_rate = CustomSchedule(d_model=embed_dim)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
transformer1.compile(optimizer =optimizer,loss = masked_loss,metrics=[masked_acc])

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │    11,883,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │    15,574,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (64, 25, 11697)        │     3,754,737 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,212,721 (119.07 MB)

 Trainable params: 31,212,721 (119.07 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
transformer1.load_weights('english_hindi_model.weights.h5')

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 345 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
transformer1.evaluate(new_val_ds)

606/606 ━━━━━━━━━━━━━━━━━━━━ 334s 551ms/step - loss: 1.4475 - masked_acc: 0.7639


[1.472076416015625, 0.760397732257843]

In [18]:
embed_dim = 128
heads = 2
latent = 512
no_of_layers = 2
vocab_size_eng = len(english_vocab)
vocab_size_hin = len(hindi_vocab)

transformer2 = Transformer(embedding_dim=embed_dim,num_heads=heads,dense_dim=latent,
                          num_layers=no_of_layers,input_vocab_size=vocab_size_eng,
                           output_vocab_size=vocab_size_hin)

In [19]:
logits2 = transformer2((inp,tar_in))
transformer2.summary()
learning_rate = CustomSchedule(d_model=embed_dim)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
transformer2.compile(optimizer =optimizer,loss = masked_loss,metrics=[masked_acc])

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

Model: "transformer_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_1 (Encoder)             │ ?                      │     1,864,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_1 (Decoder)             │ ?                      │     2,290,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (64, 25, 11697)        │     1,508,913 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,663,409 (21.60 MB)

 Trainable params: 5,663,409 (21.60 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
transformer2.load_weights("eng_hin.weights.h5")

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 177 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
transformer2.evaluate(new_val_ds)

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

606/606 ━━━━━━━━━━━━━━━━━━━━ 98s 159ms/step - loss: 1.6188 - masked_acc: 0.7519


[1.6467397212982178, 0.7485809922218323]

In [24]:
hindi_vocab = hindi_vectorization.get_vocabulary()
hindi_index_lookup = dict(zip(range(len(hindi_vocab)), hindi_vocab))
max_decoded_sentence_length = 25

def decode_sentence(model,input_sentence):
    tokenized_input_sentence = english_vectorization([input_sentence])
    decoded_sentence = "[SOS]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = hindi_vectorization([decoded_sentence])[:, :-1]
        predictions = model([tokenized_input_sentence, tokenized_target_sentence])
        # print(predictions)
        # next_tokens,next_logits = tf.nn.top_k(predictions,k = 2)
        # print(next_tokens)
        # print(next_logits)
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        # print(sampled_token_index)
        sampled_token = hindi_index_lookup[sampled_token_index]
        # print(sampled_token)
        decoded_sentence += " " + sampled_token
        if sampled_token == "[EOS]":
            break
    return decoded_sentence.lstrip("[SOS]").rstrip("[EOS]")

In [26]:
text = "Chemistry"
print(decode_sentence(transformer1,text))
print(beam_search_decoder(model =transformer1,input_sentence =text,beam_width=3,
                        english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                        hindi_index_lookup=hindi_index_lookup))
print(decode_sentence(transformer2,text))
print(beam_search_decoder(model =transformer2,input_sentence =text,beam_width=3,
                        english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                        hindi_index_lookup=hindi_index_lookup))

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

 आवाज़ 
 आवाज़ 
 वस्तु प्रतीक 
 संगीत 


In [27]:
random.seed(42)
for _ in range(5):
    input_sentence,output_sentence = random.choice(train_pairs)
    # input_sentence = input_sentence.lower()
    # input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated1 = decode_sentence(transformer1,input_sentence)
    translated2 = beam_search_decoder(model =transformer1,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    translated3 = decode_sentence(transformer2,input_sentence)
    translated4 = beam_search_decoder(model =transformer2,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    print(f"input: {input_sentence}")
    print("model1 translations")
    print(f"normal_tranlsation_     : {translated1}")
    print(f"beam_search_translation: {translated2}")
    print("model2 translations")
    print(f"normal_tranlsation_     : {translated3}")
    print(f"beam_search_translation: {translated4}")
    print(f"original__sentence     :  {output_sentence}")
    print()

input: initial file chooser folder
model1 translations
normal_tranlsation_     :  आरंभिक फाइल चयनक फ़ोल्डर 
beam_search_translation:  आरंभिक फाइल चयनक फ़ोल्डर 
model2 translations
normal_tranlsation_     :  आरंभिक फाइल चयनक फ़ोल्डर 
beam_search_translation:  आरंभिक फाइल चयनक फ़ोल्डर 
original__sentence     :  प्रारंभिक फ़ाइल चयनकर्ता फ़ोल्डर

input: so the first thing to figure out about this hyperbola is ,
model1 translations
normal_tranlsation_     :  तो पहली बात यह है के बारे में , इस अति परवलय समझ से है 
beam_search_translation:  तो पहली बात यह है के बारे में , इस अति परवलय समझ से है 
model2 translations
normal_tranlsation_     :  तो पहली बात यह है के बारे में पहली बात यह है , 
beam_search_translation:  तो पहली बात यह है के बारे में पहली बात , 
original__sentence     :  तो , पहली बात यह है के बारे में इस अति परवलय समझ से बाहर है

input: . what does right to information mean ?
model1 translations
normal_tranlsation_     :  सूचना अधिकार का क्या मतलब है ? 
beam_search_translation:  सू

In [28]:
random.seed(42)
for _ in range(5):
    input_sentence,output_sentence = random.choice(val_pairs)
    # input_sentence = input_sentence.lower()
    # input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated1 = decode_sentence(transformer1,input_sentence)
    translated2 = beam_search_decoder(model =transformer1,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    translated3 = decode_sentence(transformer2,input_sentence)
    translated4 = beam_search_decoder(model =transformer2,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    print(f"input: {input_sentence}")
    print("model1 translations")
    print(f"normal_tranlsation_     : {translated1}")
    print(f"beam_search_translation: {translated2}")
    print("model2 translations")
    print(f"normal_tranlsation_     : {translated3}")
    print(f"beam_search_translation: {translated4}")
    print(f"original__sentence     :  {output_sentence}")
    print()

input: and carry the .
model1 translations
normal_tranlsation_     :  को हासिल ले लो 
beam_search_translation:  को हासिल ले लो 
model2 translations
normal_tranlsation_     :  और आगे ले जाते है 
beam_search_translation:  और आगे ले जाते है 
original__sentence     :  और को हासिल ले लेते हैं

input: india is a country in south asia .
model1 translations
normal_tranlsation_     :  भारत देश का एक देश है दक्षिण एशिया 
beam_search_translation:  दक्षिण एशिया का एक देश है । 
model2 translations
normal_tranlsation_     :  भारत दक्षिण एशिया का भारत है । 
beam_search_translation:  भारत दक्षिण एशिया का भारत है । 
original__sentence     :  भारत देश उत्तरी गोलार्ध में स्थित है ।

input: we should first make sure that it s not in this world .
model1 translations
normal_tranlsation_     :  सबसे पहले यह इस दुनिया में नहीं है कि यह सुनिश्चित हों 
beam_search_translation:  सबसे पहले यह है कि इस संसार में नहीं है 
model2 translations
normal_tranlsation_     :  हम पहली बार को यह यकीन करना चाहिए कि ये नही है 

In [29]:
random.seed(42)
for _ in range(5):
    input_sentence,output_sentence = random.choice(test_pairs)
    # input_sentence = input_sentence.lower()
    # input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated1 = decode_sentence(transformer1,input_sentence)
    translated2 = beam_search_decoder(model =transformer1,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    translated3 = decode_sentence(transformer2,input_sentence)
    translated4 = beam_search_decoder(model =transformer2,input_sentence =input_sentence,beam_width=3,
                                      english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                                      hindi_index_lookup=hindi_index_lookup)
    print(f"input: {input_sentence}")
    print("model1 translations")
    print(f"normal_tranlsation_     : {translated1}")
    print(f"beam_search_translation: {translated2}")
    print("model2 translations")
    print(f"normal_tranlsation_     : {translated3}")
    print(f"beam_search_translation: {translated4}")
    print(f"original__sentence     :  {output_sentence}")
    print()


input: so we wanted to win this match .
model1 translations
normal_tranlsation_     :  इसलिए हम इस मैच को जीतना चाहते थे । 
beam_search_translation:  इसलिए हम इस मैच को जीतना चाहते थे । 
model2 translations
normal_tranlsation_     :  इसलिए हम इस मैच को जीतना चाहते थे । 
beam_search_translation:  इसलिए हम यह मैच जीतना चाहते थे । 
original__sentence     :  इसलिए हम इस मैच को जीतना चाहते थे .

input: it will have a common name all over the country .
model1 translations
normal_tranlsation_     :  यह देश भर में एक सामान्य नाम है । 
beam_search_translation:  यह देश भर में एक सामान्य नाम है । 
model2 translations
normal_tranlsation_     :  यह सब देश में एक ही नाम होगा । 
beam_search_translation:  देश भर में यह सब कुछ नाम होगा । 
original__sentence     :  देश भर में दुकान के लिए एक ही नाम होगा ।

input: please select a folder below
model1 translations
normal_tranlsation_     :  कृपया नीचे एक फ़ोल्डर चुनें 
beam_search_translation:  कृपया नीचे एक फ़ोल्डर चुनें 
model2 translations
normal_tranls

In [30]:
my_text = ["where are you ?",
          "i am not one of them ",
          "what do you think about them ?",
          "today is a good day",
          "who are you?",]

In [32]:
for i in my_text:
    x = decode_sentence(transformer1,i)
    y = beam_search_decoder(model =transformer1,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    x1 = decode_sentence(transformer2,i)
    y1 = beam_search_decoder(model =transformer2,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    print('sentence: ',i)
    print('model_1')
    print('normal_search_result: ',x)
    print('beam_search_result  : ',y)
    print("model_2")
    print('normal_search_result: ',x1)
    print('beam_search_result  : ',y1)
    print()

c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

sentence:  where are you ?
model_1
normal_search_result:   कहां हैं ? 
beam_search_result  :   तुम कहाँ हो ? 
model_2
normal_search_result:   तुम कहाँ हो ? 
beam_search_result  :   तुम कहाँ हो ? 

sentence:  i am not one of them 
model_1
normal_search_result:   मैं उनमें से एक नहीं हूँ 
beam_search_result  :   मैं उनमें से एक नहीं हूँ 
model_2
normal_search_result:   मैं उनमें से एक नहीं हूँ । 
beam_search_result  :   मैं उनमें से एक नहीं हूँ 

sentence:  what do you think about them ?
model_1
normal_search_result:   तुम्हारे बारे में क्या लगता है । 
beam_search_result  :   तुम्हारे बारे में क्या लगता है । 
model_2
normal_search_result:   क्या आपको उनकी बारे में क्या लगता है ? 
beam_search_result  :   उनके बारे में क्या सोच रहे हो ? 

sentence:  today is a good day
model_1
normal_search_result:   आज का दिन अच्छा है 
beam_search_result  :   आज का दिन अच्छा है 
model_2
normal_search_result:   आज अच्छा दिन है । 
beam_search_result  :   आज अच्छा दिन है । 

sentence:  who are you?
model_1
n

In [33]:
my_text_1 = ["hi my name is tom",
           "how are you people",
           "once again",
           " hello, my name is tom . how are you ?",
           "Hope today is a good day",

           ]


In [39]:

for i in my_text_1:
    x = decode_sentence(transformer1,i)
    y = beam_search_decoder(model =transformer1,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    x1 = decode_sentence(transformer2,i)
    y1 = beam_search_decoder(model =transformer2,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    print('sentence: ',i)
    print('model_1')
    print('normal_search_result: ',x)
    print('beam_search_result  : ',y)
    print("model2")
    print('normal_search_result: ',x1)
    print('beam_search_result  : ',y1)
    print()


c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\sivat\Anaconda3\envs\tensorflow-env\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

sentence:  hi my name is tom
model_1
normal_search_result:   मेरा नाम टॉम है । 
beam_search_result  :   मेरा नाम टॉम है । 
model2
normal_search_result:   लिखा हुआ नाम टॉम 
beam_search_result  :   लिखा हुआ नाम टॉम 

sentence:  how are you people
model_1
normal_search_result:   कैसे हैं जनता 
beam_search_result  :   लोग कैसे हैं 
model2
normal_search_result:   आप लोग कैसे हैं 
beam_search_result  :   आप लोग कैसे हैं 

sentence:  once again
model_1
normal_search_result:   और एक बार 
beam_search_result  :   और एक बार 
model2
normal_search_result:   एक बार फिर 
beam_search_result  :   एक बार फिर 

sentence:   hello, my name is tom . how are you ?
model_1
normal_search_result:   नमस्ते , आप टॉम का नाम है । आप कैसे हैं ? 
beam_search_result  :   नमस्ते , मेरा नाम टॉम है । आप कैसे हैं ? 
model2
normal_search_result:   नमस्ते , मेरा नाम है टॉम । आप कैसे हैं ? 
beam_search_result  :   नमस्ते , मेरा नाम है टॉम । आप कैसे हैं ? 

sentence:  Hope today is a good day
model_1
normal_search_result:   उ

In [35]:
text = "i listen to bird tweet in the morning"
print(decode_sentence(transformer1,text))
print(decode_sentence(transformer2,text))


 मैं सुबह के लिए पक्षी कुंजी की बात करें 
 मैं सुबह की नई डाक की सारी खबर सुनता हूं 


In [36]:
print(beam_search_decoder(model =transformer1,input_sentence =text,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup))
print(beam_search_decoder(model =transformer2,input_sentence =text,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup))

 मैं सुबह पक्षियों को देखे की देखे 
 मैं सुबह के समय की बिलकुल खबर सुनता हूं 


In [43]:
random.seed(24)
my_text_2 = random.sample(english_vocab,5)
for i in my_text_2:
    x = decode_sentence(transformer1,i)
    y = beam_search_decoder(model =transformer1,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    x1 = decode_sentence(transformer2,i)
    y1 = beam_search_decoder(model =transformer2,input_sentence =i,beam_width=3,
                            english_vectorization=english_vectorization,hindi_vectorization=hindi_vectorization,
                            hindi_index_lookup=hindi_index_lookup)
    print('sentence: ',i)
    print('model_1')
    print('normal_search_result: ',x)
    print('beam_search_result  : ',y)
    print("model2")
    print('normal_search_result: ',x1)
    print('beam_search_result  : ',y1)
    print()


sentence:  ported
model_1
normal_search_result:   मामले हैं पोर्टेड 
beam_search_result  :   पोर्टेड 
model2
normal_search_result:   ही पोर्टेड 
beam_search_result  :   पोर्टेड 

sentence:  alignat
model_1
normal_search_result:   लकीर 
beam_search_result  :   लकीर 
model2
normal_search_result:   लकीर 
beam_search_result  :   लकीर 

sentence:  neighborhood
model_1
normal_search_result:   पड़ोस 
beam_search_result  :   पड़ोस 
model2
normal_search_result:   पड़ोस 
beam_search_result  :   पड़ोस 

sentence:  breathe
model_1
normal_search_result:   साँस 
beam_search_result  :   साँस 
model2
normal_search_result:   साँस 
beam_search_result  :   साँस 

sentence:  eleven
model_1
normal_search_result:   ग्यारह 
beam_search_result  :   ग्यारह 
model2
normal_search_result:   स्तर 
beam_search_result  :   स्तर 



 कल प्रदर्शित प्रशिक्षण 
 कल प्रदर्शित प्रशिक्षण 
